In [32]:
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import pandas as pd
import numpy as np
nltk.download('punkt')


# Data Preperation

data = pd.read_csv('car-reviews.csv')

positive_data = data[691:]
negative_data = data[:691]

train_positive = positive_data[:553]
test_positive = positive_data[553:]

train_negative = negative_data[:553]
test_negative = negative_data[553:]

raw_training_data = pd.concat([train_positive, train_negative]).reset_index(drop=True)
raw_testing_data = pd.concat([test_positive, test_negative]).reset_index(drop=True)

raw_training_data['Sentiment'] = np.where(raw_training_data['Sentiment'] == 'Pos', 1, 0)
raw_testing_data['Sentiment'] = np.where(raw_testing_data['Sentiment'] == 'Pos', 1, 0)


# Assuming raw_training_data is your DataFrame and 'Review' is the column with text
# Tokenize each review into sentences and then each sentence into words
tagged_reviews = []
for review in raw_training_data['Review']:
    # Tokenize the review into sentences
    sentences = sent_tokenize(review)
    # Tokenize each sentence into words and append to tagged_reviews
    tagged_sentences = [[(word, 'O') for word in word_tokenize(sentence)] for sentence in sentences]
    tagged_reviews.extend(tagged_sentences)

# Now tagged_reviews contains a list of sentences, where each sentence is a list of tuples (word, 'O')
# 'O' is just a placeholder to mimic the structure you provided. You might want to replace it with actual tags.

[nltk_data] Downloading package punkt to /Users/leon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
def word2features(sent, i):
    word = sent[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        # ... other features
    }
    # Add features for previous word
    if i > 0:
        prev_word = sent[i-1][0]
        features.update({
            '-1:word.lower()': prev_word.lower(),
            '-1:word.istitle()': prev_word.istitle(),
            '-1:word.isupper()': prev_word.isupper(),
        })
    else:
        features['BOS'] = True # Beginning of sentence
    
    # Add features for next word
    if i < len(sent)-1:
        next_word = sent[i+1][0]
        features.update({
            '+1:word.lower()': next_word.lower(),
            '+1:word.istitle()': next_word.istitle(),
            '+1:word.isupper()': next_word.isupper(),
        })
    else:
        features['EOS'] = True # End of sentence
    return features

# Apply feature extraction to each sentence
X = [[word2features(s, i) for i in range(len(s))] for s in tagged_reviews]
y = [[tag for word, tag in s] for s in tagged_reviews]


In [35]:
# Split the data into training and test sets (typically you'd have more data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [41]:
# Train the CRF model
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

print(str(crf.fit(X_train, y_train)))

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

In [37]:
# Predict on the test set
y_pred = crf.predict(X_test)

# Evaluate the model
print(metrics.flat_accuracy_score(y_test, y_pred))


1.0
